# Note this code uses 

* Python 3.7
* Tensorflow 1.14.0
* HMR from [here](https://github.com/peabody124/hmr/) with a few compatibility updates

Please register and download SMPL from here https://smpl.is.tue.mpg.de

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
import cv2
import tensorflow as tf
import skimage.io as io

import os
import sys

## Set up the HMR model

In [ ]:

sys.path.append('../hmr')

from src.util import image as img_util
from src.util import openpose as op_util
import src.config
from src.RunModel import RunModel

# fake that we are running an app like in hmr/demo.py
from absl import flags
config = flags.FLAGS
config(['hmr.py'])
config.load_path = src.config.PRETRAINED_MODEL

#config.img_size = 640
config.batch_size = 1

hmr_graph = tf.get_default_graph()
with hmr_graph.as_default():
    model = RunModel(config, sess=tf.Session(graph=hmr_graph))

# Analyze frame

In [ ]:
video_fn = f'movement.mp4'
vid = cv2.VideoCapture(video_fn)
total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)
vid.release()

def get_frame(frame_id, top=True):
    vid = cv2.VideoCapture(video_fn)

    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = vid.read()

    # only keep one of the two cameras
    if top:
        frame = frame[:480]
    else:
        frame = frame[480:]
    
    return frame

In [ ]:
def preprocess_image(img, json_path=None):
    
    if json_path is None:
        if np.max(img.shape[:2]) != config.img_size:
            scale = (float(config.img_size) / np.max(img.shape[:2]))
        else:
            scale = 1.

        center = np.round(np.array(img.shape[:2]) / 2).astype(int)
        # image center in (x,y)
        center = center[::-1]
    else:
        scale, center = op_util.get_bbox(json_path)
    
    crop, proc_param = img_util.scale_and_crop(img, scale, center, config.img_size)

    # Normalize image to [-1, 1]
    crop = 2 * ((crop / 255.) - 0.5)

    return crop, proc_param, img

def get_json_path(frame_id, top=True):
    n = frame_id * 2 + top
    return f'movement{filename_N}_json/{n}_keypoints.json'

frame_id = 100
top = True

print(get_json_path(frame_id, top))
frame = get_frame(frame_id, top)
crop, proc_param, img = preprocess_image(frame, get_json_path(frame_id, top))

input_img = np.expand_dims(crop, 0)
joints, verts, cams, joints3d, theta = model.predict(input_img, get_theta=True)

plt.figure(figsize=(8,8))
plt.imshow(0.5 + 0.5 * input_img[0])
for j in joints[0]:
    plt.plot(j[0], j[1], 'ow', MarkerSize=1.0)

# Visualize with Open3D

In [ ]:
# serialization.py

import sys
sys.path.append('../smpl_modified/smpl_webuser')
import numpy as np
import _pickle as old_pickle
import chumpy as ch
from chumpy.ch import MatVecMult
from posemapper import posemap
from verts import verts_core
    
def save_model(model, fname):
    m0 = model
    trainer_dict = {'v_template': np.asarray(m0.v_template),'J': np.asarray(m0.J),'weights': np.asarray(m0.weights),'kintree_table': m0.kintree_table,'f': m0.f, 'bs_type': m0.bs_type, 'posedirs': np.asarray(m0.posedirs)}    
    if hasattr(model, 'J_regressor'):
        trainer_dict['J_regressor'] = m0.J_regressor
    if hasattr(model, 'J_regressor_prior'):
        trainer_dict['J_regressor_prior'] = m0.J_regressor_prior
    if hasattr(model, 'weights_prior'):
        trainer_dict['weights_prior'] = m0.weights_prior
    if hasattr(model, 'shapedirs'):
        trainer_dict['shapedirs'] = m0.shapedirs
    if hasattr(model, 'vert_sym_idxs'):
        trainer_dict['vert_sym_idxs'] = m0.vert_sym_idxs
    if hasattr(model, 'bs_style'):
        trainer_dict['bs_style'] = model.bs_style
    else:
        trainer_dict['bs_style'] = 'lbs'
    pickle.dump(trainer_dict, open(fname, 'w'), -1)


def backwards_compatibility_replacements(dd):

    # replacements
    if 'default_v' in dd:
        dd['v_template'] = dd['default_v']
        del dd['default_v']
    if 'template_v' in dd:
        dd['v_template'] = dd['template_v']
        del dd['template_v']
    if 'joint_regressor' in dd:
        dd['J_regressor'] = dd['joint_regressor']
        del dd['joint_regressor']
    if 'blendshapes' in dd:
        dd['posedirs'] = dd['blendshapes']
        del dd['blendshapes']
    if 'J' not in dd:
        dd['J'] = dd['joints']
        del dd['joints']

    # defaults
    if 'bs_style' not in dd:
        dd['bs_style'] = 'lbs'



def ready_arguments(fname_or_dict):

    if not isinstance(fname_or_dict, dict):
        dd = old_pickle.load(open(fname_or_dict, 'rb'), fix_imports=True, encoding='latin1')
    else:
        dd = fname_or_dict
        
    backwards_compatibility_replacements(dd)
        
    want_shapemodel = 'shapedirs' in dd
    nposeparms = dd['kintree_table'].shape[1]*3

    if 'trans' not in dd:
        dd['trans'] = np.zeros(3)
    if 'pose' not in dd:
        dd['pose'] = np.zeros(nposeparms)
    if 'shapedirs' in dd and 'betas' not in dd:
        dd['betas'] = np.zeros(dd['shapedirs'].shape[-1])

    for s in ['v_template', 'weights', 'posedirs', 'pose', 'trans', 'shapedirs', 'betas', 'J']:
        if (s in dd) and not hasattr(dd[s], 'dterms'):
            dd[s] = ch.array(dd[s])

    if want_shapemodel:
        dd['v_shaped'] = dd['shapedirs'].dot(dd['betas'])+dd['v_template']
        v_shaped = dd['v_shaped']
        J_tmpx = MatVecMult(dd['J_regressor'], v_shaped[:,0])        
        J_tmpy = MatVecMult(dd['J_regressor'], v_shaped[:,1])        
        J_tmpz = MatVecMult(dd['J_regressor'], v_shaped[:,2])        
        dd['J'] = ch.vstack((J_tmpx, J_tmpy, J_tmpz)).T    
        dd['v_posed'] = v_shaped + dd['posedirs'].dot(posemap(dd['bs_type'])(dd['pose']))
    else:    
        dd['v_posed'] = dd['v_template'] + dd['posedirs'].dot(posemap(dd['bs_type'])(dd['pose']))
            
    return dd



def load_model(fname_or_dict):
    dd = ready_arguments(fname_or_dict)
    
    args = {
        'pose': dd['pose'],
        'v': dd['v_posed'],
        'J': dd['J'],
        'weights': dd['weights'],
        'kintree_table': dd['kintree_table'],
        'xp': ch,
        'want_Jtr': True,
        'bs_style': dd['bs_style']
    }
    
    result, Jtr = verts_core(**args)
    result = result + dd['trans'].reshape((1,3))
    result.J_transformed = Jtr + dd['trans'].reshape((1,3))

    for k, v in dd.items():
        setattr(result, k, v)
        
    return result

In [ ]:
import open3d as o3d
m = load_model('../smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl')

m.pose[:] = theta[0,3:75]
m.betas[:] = theta[0, 75:]

m.pose[0] = 0
m.pose[1] = 0
m.pose[2] = 0

def pcd_from_smpl(m):
    vertices = o3d.utility.Vector3dVector(m)
    faces = o3d.utility.Vector3iVector(m.f)

    pcd = o3d.geometry.TriangleMesh()
    pcd.vertices = vertices
    pcd.triangles = faces
    pcd.compute_triangle_normals()
    pcd.compute_vertex_normals()
    
    return pcd

o3d.visualization.draw_geometries([pcd_from_smpl(m)])

# Process whole video through this pipeline

In [ ]:
from tqdm import tqdm

results_top = []
results_bottom = []

for i in tqdm(range(0, total_frames)):
    
    try:
        frame = get_frame(i, top=True)
        crop, proc_param, img = preprocess_image(frame, get_json_path(i, True))
        input_img = np.expand_dims(crop, 0)
        joints, verts, cams, joints3d, theta = model.predict(input_img, get_theta=True)    
        res_top = {'joints': joints[0], 'verts': verts[0], 'cams': cams[0], 'joints3d': joints3d[0], 'theta': theta[0,3:75], 'beta': theta[0,75:]}
    except:
        print('skip')
    results_top.append(res_top)
    
    try:
        frame = get_frame(i, top=False)
        crop, proc_param, img = preprocess_image(frame, get_json_path(i, False))
        input_img = np.expand_dims(crop, 0)
        joints, verts, cams, joints3d, theta = model.predict(input_img, get_theta=True)
        res_bottom = {'joints': joints[0], 'verts': verts[0], 'cams': cams[0], 'joints3d': joints3d[0], 'theta': theta[0,3:75], 'beta': theta[0,75:]}
    except:
        print('skip')
    results_bottom.append(res_bottom)

In [ ]:
with open (f'results{filename_N}.pkl', 'wb') as handle:
    pickle.dump([results_top, results_bottom], handle, protocol=pickle.HIGHEST_PROTOCOL)